# Personal Running Code

## Pipeline

### Scanning documents

In [ ]:
# --- Quiet, progress-friendly PDF loader ---
import time, logging, warnings, contextlib, io
from pathlib import Path
from tqdm import tqdm

# 1) Silence pypdf chatter/warnings
try:
    from pypdf.errors import PdfReadWarning
    warnings.filterwarnings("ignore", category=PdfReadWarning)
except Exception:
    pass  # older pypdf versions may not expose PdfReadWarning

logging.getLogger("pypdf").setLevel(logging.ERROR)

# 2) Load PDFs with a progress bar (no ipywidgets required)
from langchain.document_loaders import PyPDFLoader

pdf_dir = Path("data")
pdf_paths = sorted(pdf_dir.rglob("*.pdf"))

docs = []
start = time.perf_counter()

print(f"Scanning {len(pdf_paths)} PDFs in {pdf_dir.resolve()} ...")
for p in tqdm(pdf_paths, desc="Loading PDFs", unit="file"):
    try:
        # Some libraries print to stderr; swallow it to keep the notebook clean
        with contextlib.redirect_stderr(io.StringIO()):
            loader = PyPDFLoader(str(p))
            docs.extend(loader.load())
    except Exception as e:
        print(f":warning: Skipped {p.name}: {e}")

elapsed = time.perf_counter() - start
print(f"Done. Loaded {len(docs)} document chunks from {len(pdf_paths)} PDF files in {elapsed:,.1f}s.")

### Pipeline

In [ ]:
# --- Split (unchanged) ---
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(docs)
print(f"Chunks after splitting: {len(split_docs)}")

# --- Embeddings (new import path) ---
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True}  # IP == cosine
)

# --- In-memory vector store (Chroma) ---
from langchain_community.vectorstores import Chroma  # works with LangChain 0.2+

# build an in-memory index from your split_docs
db = Chroma.from_documents(split_docs, embeddings)   # no persist_directory -> RAM only
retriever = db.as_retriever(search_kwargs={"k": 5})

print("Chroma (in-memory) vector store ready ✅")


# --- Retriever (choose one) ---
retriever = db.as_retriever(search_kwargs={"k": 5})
# retriever = db.as_retriever(search_type="mmr",
#                             search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5})

# --- QA chain (HF example; swap if needed) ---
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import RetrievalQA

llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-base",
    temperature=0,
    max_new_tokens=512,
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)
print("Pipeline ready ✅")

### Smaller chunks

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Split into chunks of ~1000 characters with 200-character overlap
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

print(f"Total chunks after splitting: {len(split_docs)}")
print("\n--- Preview of first chunk ---\n")
print(split_docs[0].page_content[:500])

### Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

### Vectors

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(split_docs, embeddings)
retriever = db.as_retriever()

## Test Cases

### UC1: Find Particular Documents

In [ ]:
query = "Where is the CAPP final report from 2024?"
result = qa_chain({"query": query})

print("\n--- UC1: Find Particular Documents ---\n")
print("Response:", result["result"], "\n")

print("Documents used:")
for doc in result["source_documents"]:
    print(f"{doc.metadata.get('source', 'unknown')} \n{doc.page_content[:200]}...\n")


UC2: Summarize Particular Documents

In [ ]:
query = "Summarize the CAPP final report from 2024"
result = qa_chain({"query": query})

print("\n--- UC2: Summarize Particular Documents ---\n")
print("Summary:", result["result"], "\n")

print("Documents used:")
for doc in result["source_documents"]:
    print(f"{doc.metadata.get('source', 'unknown')}\n")


UC3: Find Documents by Contents

In [ ]:
query = "Find documents related to system executive policies on AI"
result = qa_chain({"query": query})

print("\n--- UC3: Find Documents by Contents ---\n")
print("Response:", result["result"], "\n")

print("Documents used:")
for doc in result["source_documents"]:
    print(f"{doc.metadata.get('source', 'unknown')}\n")


UC4: Finding Particular Information

In [ ]:
query = "When were votes on AI policies conducted?"
result = qa_chain({"query": query})

print("\n--- UC4: Finding Particular Information ---\n")
print("Response:", result["result"], "\n")

print("Documents used:")
for doc in result["source_documents"]:
    print(f"{doc.metadata.get('source', 'unknown')}\n")


UC5: Finding Related Information

In [ ]:
query = "Show me the history of resolutions on GE"
result = qa_chain({"query": query})

print("\n--- UC5: Finding Related Information ---\n")
print("Response:", result["result"], "\n")

print("Documents used:")
for doc in result["source_documents"]:
    print(f"{doc.metadata.get('source', 'unknown')}\n")


UC6: Refinement of Found Information

In [ ]:
query = "In the history of resolutions you showed me, which ones are supportive or opposing GE reforms?"
result = qa_chain({"query": query})

print("\n--- UC6: Refinement of Found Information ---\n")
print("Response:", result["result"], "\n")

print("Documents used:")
for doc in result["source_documents"]:
    print(f"{doc.metadata.get('source', 'unknown')}\n")
